# Lab 2

## Inception

**Трішки попереджень.** Протягом всієї лабки буде дуже багато тексту й пояснень і взагалі мого розуміння того, що відбувається. І в різні етапи виконання роботи поставали різні питання. Десь деякі із них наведені в процесі виконання роботи, проте я б хотів виокремити головні із них тут і був би дуже вдячним, якби отримав на них відповідь :). І взагалі я готовий до критики й мені цікаво, чи я не даремно витратив кілька днів на реалізацію цієє лабки.
- **Чи правильно було вибране еталонне зображення?**
- **Чи правильне рішення щодо метрик?**
- **Що ж таке "відносна кількість правильно суміщених ознак" на прикладі мого отриманого датафрейму?**
- **Чи правильний набір тестових зображень?**
- **Чи адекватні результати тестового набору зображень?**

**Ну що ж, розпочнімо.** Для тестування я вибрав **дескриптор ORB**, а **предмет - BMW X6**. Не хвилюйтеся, це не просто картинки машини із Інтернету, це її досить невеличка моделька, яка є в мене вдома. Із цього ж моменту почалися деякі труднощі, оскільки майже у всіх туторіалах і зразках коду, які я знаходив, є деяке "еталонне" зображення, з яким і порівнюють усі тестові (найчастіше еталонним виступає картинка титульної сторінки книжки, з якої можна знайти ну дужееее багато фіч і з якою круто порівнювати вже всі інші зображення, зроблені власноруч). Тому резонне запитання: "Яке ж зображення взяти за еталонне в цьому випадку?" Було вирішено взяти ось таке зображення:

<img src="main_photo.jpg" alt="Drawing" style="width: 400px;"/>

Чому таке? Воно зроблене під певним кутом, де видно повністю один бік, перед, верх і навіть трішки задню фару, тому мало б у більшості випадків знаходити спільні фічі із цією ж машинкою, сфотографованою під іншим ракурсом або кутом. Проте зрозуміло, що якщо буде зроблена фотографія із іншого боку, ззаду, зверху, 4 виміру, або ж взагалі під якимось неприробним кутом, програмі буде дуже важко знайти багато спільних фіч. Можливо, моя логіка й не надто правильна, але це ж лише моя перша подібна програма, тому не судіть надто строго :)

Наступна частина викладок буде після основної частини коду, вчитатися у який трішки складно. Або ж можна просто заранити його й проскролити трішки вниз (а можливо, і не трішки).

## Import libraries

In [11]:
import cv2
import numpy as np
import pandas as pd
import time
import os

## Define class for object recognition

In [12]:
class ObjectRecignition:
    """
    Arguments:
        path_main: string -- path for main (train) image
        directory_test: string -- directory with all images for testing
        directory_save: string -- directory where you want to save all possible savings
        n_features: int -- number of features that will be used as a parameter for ORB descriptor (default: 1500)
    """
    
    def __init__(self, path_main, directory_test, directory_save, n_features=1500):
        self.path_main = path_main
        self.directory_test = directory_test
        self.directory_save = directory_save
        self.n_features = n_features
        self.__orb = cv2.ORB_create(nfeatures=n_features)
        self.img_main = cv2.imread(path_main, cv2.IMREAD_GRAYSCALE)
        self.keypoints_main, self.descriptors_main = self.__orb.detectAndCompute(self.img_main, None)
        self.__bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    
    def get_metrics(self, path):
        """
        Return metrics for main and another image
        Arguments:
            path: string -- path for another image
        Returns:
            features: int -- number of features
            all_matches: int -- number of all matches
            true_matches: int -- number of true matches (find by findHomography)
            error_all_matches: float -- mean of distances of DMatch objects for all matches
            error_true_matches: float -- mean of distances of DMatch objects for true matches
            size: tuple -- size of image
            time: float -- time of running the function
        """        
        # Initialize an image
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        kp, des = self.__orb.detectAndCompute(img, None)
        
        # Find features
        features = self.n_features
        
        # Find time
        start_time = time.time()
        
        # Find all_matches
        try:
            matches = self.__bf.match(self.descriptors_main, des)
        except:
            print("Something wrong with image", path)
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
        matches = sorted(matches, key = lambda x: x.distance)
        all_matches = len(matches)
        
        # Find true_matches
        query_pts = np.float32([self.keypoints_main[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        train_pts = np.float32([kp[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        _, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()
        true_matches_list = []
        for index, el in enumerate(matches_mask):
            if el == 1:
                true_matches_list.append(matches[index])
        true_matches = len(true_matches_list)
        
        # Find time
        end_time = time.time()
        time_ = round(end_time - start_time, 4)
        
        # Find error_all_matches
        if all_matches == 0:
            error_all_matches = np.nan
        else:
            error_all_matches_list = []
            for m in matches:
                error_all_matches_list.append(m.distance)
            error_all_matches = round(np.array(error_all_matches_list).mean(), 4)
        
        # Find error_true_matches
        if true_matches == 0:
            error_true_matches = np.nan
        else:
            error_true_matches_list = []
            for m in true_matches_list:
                error_true_matches_list.append(m.distance)
            error_true_matches = round(np.array(error_true_matches_list).mean(), 4)
        
        # Find size
        size = img.shape
        
        # Здається, що алгоримт завжди знаходить у районі 10 true_matches, тому будемо вважати 10 еквівалентно 0
#         if true_matches < 10:
#             true_matches = 0
#             error_true_matches = np.nan
        
        # Return values as tuple
        return features, all_matches, true_matches, error_all_matches, error_true_matches, size, time_
    
        
    def get_all_metrics_as_df(self, print_results=False):
        """
        Return all metrics for all images from directory_test as pandas DataFrame
        Returns:
            df: pandas DataFrame -- a dataframe with all metrics for all images
        """
        all_metrics = []
        
        for filename in os.listdir(self.directory_test):
            path = self.directory_test + '\\\\' + filename
            temp_list = list(self.get_metrics(path))
            temp_list.insert(0, filename)
            all_metrics.append(temp_list)
            
        df = pd.DataFrame(all_metrics, columns=['name', 'features', 'all_matches', 'true_matches', 
                                                'error_all_matches', 'error_true_matches', 'size', 'time'])
        return df
        
    
    def save_all_metrics(self, file_name):
        """
        Save all metrics for all images as csv file
        Arguments:
            file_name: string -- name of the file (without the format)
        """
        df = self.get_all_metrics_as_df()
        df.to_csv(self.directory_save + '\\\\' + file_name + '.csv') 
    
    
    def show_features(self, save=False):
        """
        Show features on the main image
        Arguments:
            save: bool -- save received image or not (default: False)
        """
        img_keys = cv2.drawKeypoints(self.img_main, self.keypoints_main, None)
        cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Image", 600, 600)
        cv2.imshow("Image", img_keys)
        
        if save:
            file_name = self.directory_save + '\\\\' + 'features_for_' + self.path_main.split('\\')[-1]
            cv2.imwrite(file_name, img_keys)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    
    def show_all_matches(self, random=True, path='', save=False):
        """
        Show all matches between main and another image
        Arguments:
            random: bool -- show all matches for random image if random set to True (default: True)
            path: string -- path for another image if random set to False (default: empty string)
            save: bool -- save received image or not (default: False)
        """
        if random:
            path = self.directory_test + '\\\\' + np.random.choice(os.listdir(self.directory_test))
        else:
            if path == '':
                return "Please enter the path or set random to True"
            
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        kp, des = self.__orb.detectAndCompute(img, None)
        
        matches = self.__bf.match(self.descriptors_main, des)
        matches = sorted(matches, key = lambda x: x.distance)
        
        matching_result = cv2.drawMatches(self.img_main, self.keypoints_main, img, kp, matches, None)
        
        cv2.namedWindow("Matches", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Matches", 1200, 600)
        cv2.imshow("Matches", matching_result)
        
        if save:
            file_name = self.directory_save + '\\\\' +  'all_matches_for_' + path.split('\\')[-1]
            cv2.imwrite(file_name, matching_result)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    
    def show_true_matches(self, random=True, path='', save=False):
        """
        Show true matches between main and another image (finding by findHomography)
        Arguments:
            random: bool -- show all matches for random image if random set to True (default: True)
            path: string -- path for another image if random set to False (default: empty string)
            save: bool -- save received image or not (default: False)
        """
        if random:
            path = self.directory_test + '\\\\' + np.random.choice(os.listdir(self.directory_test))
        else:
            if path == '':
                return "Please enter the path or set random to True"
            
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        kp, des = self.__orb.detectAndCompute(img, None)
        
        matches = self.__bf.match(self.descriptors_main, des)
        matches = sorted(matches, key = lambda x: x.distance)
        
        matching_result = cv2.drawMatches(self.img_main, self.keypoints_main, img, kp, matches, None)
        
        query_pts = np.float32([self.keypoints_main[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        train_pts = np.float32([kp[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        _, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()
        
        true_matches = []
        for index, el in enumerate(matches_mask):
            if el == 1:
                true_matches.append(matches[index])
                
        matching_true_relults = cv2.drawMatches(self.img_main, self.keypoints_main, img, kp, true_matches, None)
        
        cv2.namedWindow("True Matches", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("True Matches", 1200, 600)
        cv2.imshow("True Matches", matching_true_relults)
        
        if save:
            file_name = self.directory_save + '\\\\' + 'true_matches_for_' + path.split('\\')[-1]
            cv2.imwrite(file_name, matching_true_relults)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()

## Explain how it works

**Напевно, зараз найбільш резонним було б пояснити, як програма взагалі працює і який її функціонал.**

Ну, як уже й було сказано, я розглядаю дексриптор ORB, який дуже просто викликаю із бібліотеки cv2. Тут же необхідно наголосити, що як параметр я передаю скільки фіч я хочу, щоб дескриптор знаходив на еталонному фото й потім на всіх тестових. За замовчуванням цей параметр дорівнює 1500. І стільки ж фіч буду використовувати я у своїх подільших дослідженнях. Потім знаходжу кількість матчів за допомогою BFMatcher і кількість "правильних" матчів за допомогою findHomography. Також обчислюю похибки локалізації окремо для всіх матчів та "правильних" матчів, як середнє значень distance об'єкту DMatch. Також обчислюю розмір тестового зображення й час роботи, необхідний на виконання того, щоб заранити дескриптор, заматчити два зображення й знайти правильні матчі. Впринципі, я пояснив роботу основної функції get_metrics() мого класу ObjectRecignition. Також пояснив, яким же чином я знаходив певні необхідні метрики, проте про них більш детально трішки згодом.

**Щоб запустити програму, користувач повинен створити об'єкт класу ObjectRecignition із такими параметрами:**
- **path_main** - шлях до зображення, яке буде використовуватися, як еталонне (із яким будуть порівнюватися всі тестові зображення);
- **directory_test** - шлях до папки, де знаходяться всі тестові зображення;
- **directory_save** - шлях до папки, у яку ви хочете зберігати всі вихідні дані, отримані в ході програми.

**Тепер щодо функціоналу:**
- **get_metrics(path)** - користувач може отримати набір метрик для еталонного та **одного** тестового зображення, передавши його шлях, як параметр (цей метод є кістяком всієї програми, проте користувач буде використовувати його вкрай рідко);
- **get_all_metrics_as_df()** - повертає датафрейм із всіма метриками для всіх тестових зображень (впринципі основний метод, який користувач буде використовувати найчастіше);
- **save_all_metrics(file_name)** - зберігає датафрейм із всіма метриками для всіх тестових зображень у форматі csv;
- **show_features(save)** - показує знайдені фічі для об'єкту класу ObjectRecignition, тобто для еталонного зображення; у користувача також є можливість зберегти отримане фото у папку directory_save, передавши save=True;
- **show_all_matches(random, path, save)** - показує всі матчі для еталонного та рандомного зображення (якщо random=True) або для еталонного та конкретного зображення, шлях якого потрібно передати у змінну path (якщо random=False); у користувача також є можливість зберегти отримане фото у папку directory_save, передавши save=True;
- **show_true_matches(random, path, save)** - показує "правильні" матчі для еталонного та рандомного зображення (якщо random=True) або для еталонного та конкретного зображення, шлях якого потрібно передати у змінну path (якщо random=False); у користувача також є можливість зберегти отримане фото у папку directory_save, передавши save=True;

## Initialize veriables

In [13]:
path_main = 'main_photo.jpg'
directory_test = 'Library'
directory_save = 'Storage'

recognition = ObjectRecignition(path_main, directory_test, directory_save)

## Show and save features on the main image

**Заранимо наступний код і подивимося, які ж фічі знаходить наш дескриптор на еталонному фото**

In [14]:
recognition.show_features(save=True)

<img src="Storage\features_for_main_photo.jpg" alt="Drawing" style="width: 400px;"/>

Оскільки, це лише моя перша подібна програма, то я навіть не знаю, чи правильне еталонне зображення я обрав і чи програма знаходить необхідну кількість фіч для еталонного зображення. Проте будемо користуватися саме цим зображенням, бо, якщо чесно, то вже трішки ліньки, після всього цього написаного коду й продуманої логіки роботи програми, вибирати новий предмет і придумувати для нього своє еталонне зображення. Я відкритий до будь-якої критики й мені було б дуже цікаво послухати думку більш освіченої людини у цій сфері. 

## Show metrics for some testing photos

Чесно кажучи, я спочатку писав код і запускав його на деякому тестовому наборі даних, який складався із 5 фото. Давайте поглянемо на цей набір і на результати роботи програми.

**Опишемо й виведемо кожне із тестових зображень у такому ж порядку, у якому вони будуть зустрічатися в кінцевому датафреймі:**
- **1 фото** - той самий бік машинки, що й на еталонному зображенні, проте трішки під іншим кутом;
- **2 фото** - зовсім інший ракурс із іншим боком машинки;
- **3 фото** - зовсім інша машинка;
- **4 фото** - дві машинки (така, як на еталонному й зовсім інша);
- **5 фото** - зовсім без нічого (тільки рожевий фон крісла).

<img src="Library\1.jpg" alt="Drawing" style="width: 400px;"/>
<img src="Library\2.jpg" alt="Drawing" style="width: 400px;"/>
<img src="Library\3.jpg" alt="Drawing" style="width: 400px;"/>
<img src="Library\4.jpg" alt="Drawing" style="width: 400px;"/>
<img src="Library\5.jpg" alt="Drawing" style="width: 400px;"/>

**Тепер заранимо наступний код і подивимося на результати всіх метрик для цих тестових зображень**

In [15]:
df = recognition.get_all_metrics_as_df()

Something wrong with image Library\\photo_2020-10-03_17-27-11.jpg
Something wrong with image Library\\photo_2020-10-03_17-27-13.jpg


In [16]:
df.head()

,name,features,all_matches,true_matches,error_all_matches,error_true_matches,size,time
0,1.jpg,1500.0,357.0,59.0,42.0868,32.1695,"(1276, 1276)",0.0898
1,2.jpg,1500.0,276.0,15.0,49.5109,35.2667,"(1276, 1276)",0.0838
2,3.jpg,1500.0,301.0,10.0,53.9369,47.2000,"(1276, 956)",0.0838
3,4.jpg,1500.0,415.0,80.0,42.8627,35.1375,"(1276, 1276)",0.0848
4,5.jpg,1500.0,290.0,8.0,61.6483,54.3750,"(1276, 1276)",0.0728


Саме такий вигляд матиме кінцевий датафрейм із усіма знайденими метриками для тестових зображень. Дивлячись на нього, у Вас може постати питання, а де ж відсотки, де перша метрика? А все просто, її й немає :). Я дійсно дуже довго намагався зрозуміти, що саме означає "відносна кількість правильно суміщених ознак". Чи це відношення true_matches до features, чи це відношення true_matches до all_matches. Тому я вирішив просто записати всі ці змінні до датафрейму, а потрібне відношення для майбутніх досліджень можна буде дуже просто знайти за допомогою однієї лінії коду.

**Тепер перейдемо до аналізу отриманих результатів (запишемо їх у формі тез):**
- спочатку наголосимо таке: наша машинка була тільки на фото номер 1, 2 та 4. І як ми бачимо, саме на цих зображеннях програма знаходить найбільшу кількість "правильних" матчів і їх найменшу похибку локалізації;
- цікаво, що найбільшу кількість усіх матчів і "правильних" матчів програма знаходить для фото, де було дві машинки (трішки пізніше ми поглянемо на ці матчі);
- із негативного: програма погано матчить машинку, сфотографовану зовсім із іншого боку й іншим кутом, тільки 15 "правильних" матчів (трішки пізніше ми також подивимося на ці матчі);
- цікавим є те, що програма знаходить 8 "правильних матчів" із просто рожевим фоном крісла, але, як я зрозумів (трішки окремо додатково потестивши програму), findHomography завжди залишає найбільш схожі фічі, тому, якщо "правильних" матчів знайдено менше або рівно 10 можна інтерпретувати це як 0 (у реалізації класу є навіть закоментований фрагмент коду, де це реалізовано);
- загалом програма працює задовільно (на мій скромний погляд), для більш детального аналізу необхідно ранити програму на більш великому наборі тестових зображень, проте зрозуміло, що ми не будемо всі із 100 із чимось фото вручно передивлятися й оприділяти, де була машинка, під яким ракурсом, із якою якістю фото й тд.

**Тепер заранимо наступний код і подивимося, як програма матчить деякі тестові зображення (а саме зображення під номерами 2 та 4)**

In [17]:
recognition.show_all_matches(random=False, path='Library\\2.jpg', save=True)
recognition.show_true_matches(random=False, path='Library\\2.jpg', save=True)

In [18]:
recognition.show_all_matches(random=False, path='Library\\4.jpg', save=True)
recognition.show_true_matches(random=False, path='Library\\4.jpg', save=True)

- **Поглянемо спочатку на матчі зображення під номером 2.** Спочатку виведемо всі матчі, а потім лише "правильні".

<img src="Storage\all_matches_for_2.jpg" alt="Drawing" style="width: 800px;"/>
<img src="Storage\true_matches_for_2.jpg" alt="Drawing" style="width: 800px;"/>

Як ми бачимо, загалом матчів дуже багато й, на перший погляд, "правильних" має бути набагато більше, ніж 15, проте зрозуміло, що машинка - симетрична й програма матчить подібні фічі для іншого боку машинки, який формально й фактично не є тим боком машинки, що й не еталонному зображенні. Ось це і є проблема об'ємних однотонних фігур для розпізнавання. Ну принаймні я на даному етапі свого поглиблення в Розпізнавання Образів не знаю, як це обійти. Саме через те такий поганий результат "подібних матчів" (хоча, можливо, цей результат і не є поганим, лише я так думаю, оскільки похибка локалізації для "правильних матчів" доволі невелика). Впринципі, видно, що всі "правильні" матчі спираються на значок BMW на капоті машинки. Так що було б набагато краще, якби машинка мала яийсь рисунок на капоті, або даху, проте таке життя й вона зовсім однотонна (біла).

- **Тепер поглянемо на матчі зображення під номером 4.** Спочатку виведемо всі матчі, а потім лише "правильні".

<img src="Storage\all_matches_for_4.jpg" alt="Drawing" style="width: 800px;"/>
<img src="Storage\true_matches_for_4.jpg" alt="Drawing" style="width: 800px;"/>

У цьому випадку, взагалі немає "правильних" матчів із іншою машинкою, а із еталонною - дужееее багато (ну в порівнянні із іншими тестовими зображеннями). Це пояснюється тим, що наша BMW знаходиться точно під таким же ракурсом, що й на еталонному зображенні. Цікаво, що фічі на передніх колесах не розуміються програмою, як "правильні", проте це вже питання до findHomography. Можливо, це пов'язано із тим, що вони занадто далеко від розміщення коліс на еталонному зображенні й трішки іншого розміру.

## Show all metrics for all images

**Ну що ж, прийшов час запустити нашу програму на всьому нашому датасеті із тестових зображень**

Заранимо наступний код і подивимось на результати. Звісно, такого аналізу, як було проведений вище, ми вже зробити не зможемо, але просто побачити загальну картину - запросто.

In [19]:
df = recognition.get_all_metrics_as_df()

Something wrong with image Library\\photo_2020-10-03_17-27-11.jpg
Something wrong with image Library\\photo_2020-10-03_17-27-13.jpg


In [20]:
df

,name,features,all_matches,true_matches,error_all_matches,error_true_matches,size,time
0,1.jpg,1500.0,357.0,59.0,42.0868,32.1695,"(1276, 1276)",0.0858
1,2.jpg,1500.0,276.0,15.0,49.5109,35.2667,"(1276, 1276)",0.0838
2,3.jpg,1500.0,301.0,10.0,53.9369,47.2000,"(1276, 956)",0.0838
3,4.jpg,1500.0,415.0,80.0,42.8627,35.1375,"(1276, 1276)",0.0848
4,5.jpg,1500.0,290.0,8.0,61.6483,54.3750,"(1276, 1276)",0.0738
...,...,...,...,...,...,...,...,...
118,photo_2020-10-03_17-26-05 (2).jpg,1500.0,359.0,27.0,46.3231,34.0370,"(1276, 956)",0.0848
119,photo_2020-10-03_17-26-05.jpg,1500.0,37.0,11.0,57.5676,51.0000,"(1276, 956)",0.0199
120,photo_2020-10-03_17-27-11.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,photo_2020-10-03_17-27-12.jpg,1500.0,185.0,9.0,61.3730,65.7778,"(1276, 956)",0.0638


**Трішки аналізу отриманих результатів**
- програма непогано (я б сказав, що навіть чудово) розпізнає машинку, яка розташована в такому ж ракурсі, що й на еталонному зображенні (тобто стоїть ближче до нас лівим боком і передом), навіть якщо є якісь візуальні перешкоди (палець, мініатюрний замок, чи інша машинка збоку) або машинка обрізана й не вся знаходиться в кадрі;
- програма непогано розпізнає машинку, яка знаходиться в такому ж ракурсі, як на еталонному фото, проте сфотографована під іншим кутом, або із іншої відстані;
- програма погано адаптована до поворотів машинки (якщо машинка стоятиме в кадрі іншим боком, задом, якщо сфотографувати машинку суто зверху);
- програма чудово не розпізнає інші об'єкти (із якихось причин навіть не матчить деякі зображення, якщо чесно навіть не знаю чому, але в такому разі програма видасть відповідне повідомлення, а до датафрейму занесе рядок NaN для цього фото);
- програма погано розпізнає машинку, якщо розмір тестового й еталонного зображення доволі різний;
- програма непогано розпізнає машинку, якщо зображення засвітлене;
- програма погано розпізнає машинку, якщо зображення зетемнене, або розмите;

**Трішки висновків**
- На мою думку, програма працює непогано й досить часто розпізнає машинку.
- Як уже було сказано раніше, таке враження, що findHomography мусить повернути в районі 10 матчів, тому там, де менше або рівне 10 "правильних матчів" можна інтрепретувати, як 0, що означає, що на фото немає нашої машинки, або вона розташована в кадрі іншим боком/задом/сфотографована зверху (хоча несправді це досить строге зауваження).
- Також я вважаю, що найкращою метрикою є кількість "правильних" матчів, оскільки вся кількість матчів - взагалі якась дика, навіть для простого зображення тільки із рожевим фоном крісла програма знаходить близько 200 матчів, а похибка локалізації - не дуже хороша метрика, бо буває таке, що зображення повернуте на кілька градусів і в цьому випадку похибка локалізації буде доволі велика, проте наша машинка розташована так само, як на еталонному зображенні, просто сфотографована під іншим кутом.
- Можливо, "відносна кількість правильно суміщених ознак", проте знати б, яких змінних це відношення :)
- Щодо часу, то я щось і не дуже зрозумів навіщо цієї метрики, бо від розміру зображення він не дуже й залежить (можливо, малося на увазі від кількості фіч для дескриптора (я пробував ранити програму на 5000 фіч, тоді одне зображення обробляється приблизно 0.5 с), але тоді потрібно було б робити різну кількість фіч для еталонного й тестового зображення, що не дуже логічно.

Наступний код написаний для того, щоб показати "правильні" матчі для якогось тестового зображення, назву якого можна легко отримати із датафрейму вище. У такому разі передаємо random=False і в змінну path передаємо шлях до файлу (не забуваємо, що це має бути повний шлях до файлу, а не лише його назва!). Або ж можна вивести "правильні" матчі для рандомного тестового зображення. У такому разі передаємо лише random=True.

In [21]:
recognition.show_true_matches(random=False, path='Library\\photo_2020-10-03_17-25-26.jpg')

## Save all metrics for all images

**Тепер збережемо отриманий датафрейм у файл для подальших досліджень**

In [22]:
recognition.save_all_metrics('my_metrics')

Something wrong with image Library\\photo_2020-10-03_17-27-11.jpg
Something wrong with image Library\\photo_2020-10-03_17-27-13.jpg


**Ну що ж, на цьому аналіз мого тестового набору добіг кінця.** Залишилось заранити цю програму на датасеті із тестових зображень для іншого предмету (датасет мого партнера по команді).

## Run my program with another dataset

In [23]:
path_main = 'friend_main_photo.jpg'
directory_test = 'Friend_Library'
directory_save = 'Friend_Storage'

recognition_friend = ObjectRecignition(path_main, directory_test, directory_save)

In [24]:
df = recognition_friend.get_all_metrics_as_df()

Something wrong with image Friend_Library\\200089900711_168583.jpg
Something wrong with image Friend_Library\\200090000526_170543.jpg
Something wrong with image Friend_Library\\200092000278_140348.jpg
Something wrong with image Friend_Library\\200108800307_88020.jpg
Something wrong with image Friend_Library\\200123300440_83286.jpg


In [25]:
df

,name,features,all_matches,true_matches,error_all_matches,error_true_matches,size,time
0,200018300047_463343.jpg,1500.0,365.0,10.0,61.2685,68.6000,"(1280, 960)",0.0858
1,200018300204_458970.jpg,1500.0,365.0,9.0,56.8740,55.4444,"(1280, 960)",0.0848
2,200018300634_461451.jpg,1500.0,373.0,9.0,58.0912,61.5556,"(960, 1280)",0.0858
3,200018500242_464323.jpg,1500.0,386.0,11.0,60.0026,59.0909,"(960, 1280)",0.0858
4,200018500302_461154.jpg,1500.0,369.0,12.0,59.1355,57.1667,"(960, 1280)",0.0848
...,...,...,...,...,...,...,...,...
139,200123300440_83286.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,200124000360_78290.jpg,1500.0,417.0,10.0,60.5779,61.5000,"(960, 1280)",0.0898
141,200124000930_79039.jpg,1500.0,362.0,13.0,62.3508,44.8462,"(612, 1280)",0.0957
142,200124100091_78760.jpg,1500.0,388.0,9.0,56.5232,56.1111,"(1280, 1280)",0.0888


Ось тут уже програма працює ще гірше, ніж на мому датасеті тестових зображень. Можливо, це пов'язано із тим, що погано підібране еталонне зображення, або ж погано знайдені фічі на еталонному/тестовому зображенні. Хоча і є декілька непоганих сходжень, зокрема програма непогано матчить сходження по напису внизу корабля. У нашому випадку програма круто матчила по значку BMW на капоті машинки, тому можна зробити сміливий висновок, що дескриптор ORB найкраще підходить для плоских зображень із написами або малюночками. У такому випадку ідеально б за еталонне зображення підійшла якась картинка титульної сторінки книжки, а для тестових зображень - ціла купа зображень із цією книжкою в руках, або ж просто її зображення під різними ракурсами, освітленням і тд.

<img src="friend_main_photo.jpg" alt="Drawing" style="width: 400px;"/>
<img src="Friend_Storage\features_for_friend_main_photo.jpg" alt="Drawing" style="width: 400px;"/>
<img src="Friend_Storage\true_matches_for_200115100733_72520.jpg" alt="Drawing" style="width: 800px;"/>

In [26]:
recognition_friend.save_all_metrics('friend_metrics')

Something wrong with image Friend_Library\\200089900711_168583.jpg
Something wrong with image Friend_Library\\200090000526_170543.jpg
Something wrong with image Friend_Library\\200092000278_140348.jpg
Something wrong with image Friend_Library\\200108800307_88020.jpg
Something wrong with image Friend_Library\\200123300440_83286.jpg
